使用ライブラリ

In [1]:
import pandas as pd         #データフレーム作成用ライブラリ
import apache_log_parser    #アクセスログ解析用ライブラリ
import collections          #集計用ライブラリ
from pprint import pprint   #表示用ライブラリ


・/var/log/httpd/access_log = アクセスログの場所

・アクセスログの構成<br>
**%h** = **10.2.3.4**:リモートホスト名<br>
**%l** = **-**:クライアントの識別子<br>
**%u** = **-**:認証ユーザー名<br>
**%t** = **[18/Apr/2005:00:10:47 +0900]**:リクエストを受信した時刻([day/month/year:hour:minute:secondzone]の書式)<br>
**\"%r\"** = **"GET / HTTP/1.1"**:リクエストの最初の行<br>
**%>s** = **200 854**:最後のレスポンスのステータス<br>
**%b** = **"-"**:HTTPヘッダを除くレスポンスのバイト数。0バイトの場合は「-」と表示される<br>
**\"%{Referer}i\"** = **"Mozilla/4.0"**:サーバが受信したリクエストヘッダのReferer<br>
**\"%{User-Agent}i\"** = **"(compatible; MSIE 5.5; Windows 98)"**:サーバが受信したリクエストヘッダのUser-Agent<br>

ex)10.2.3.4 - - [18/Apr/2005:00:10:47 +0900] "GET / HTTP/1.1" 200 854 "-""Mozilla/4.0 (compatible; MSIE 5.5; Windows 98)"

In [2]:
line_parser = apache_log_parser.make_parser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")   #アクセスログを指定された形でパースする為の設定

In [3]:
day_list = []     #日にちを分割するためのリスト
month_list = []   #月を分割するためのリスト
year_list = []    #年を分割するためのリスト
hour = [0] * 24   #時間ごとのアクセス件数を格納するリスト
host = []         #ホスト名を格納するリスト

#ログのアクセスログを読み、ホスト名、アクセス時間、日にち、月、年を各リストに登録する。最後にアクセスログを表示する
def read_apache_log(access_log, logformat="%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\""):
    parser = apache_log_parser.make_parser(logformat)
    P = []   #指定通りの形にパースできた場合こちらに格納する
    E = []   #指定通りの形にパースでき無かった場合こちらに格納する
    for i in range(100):
        try:
            with open('access_log{}'.format(i + 1)) as f:                                       #(問2)複数ファイルの対応　access_log1を1ずつ増やしていくex)access_log2,access_log99
                for line in f:
                    try:
                        parsed_line = parser(line)                                              #アクセスログをパースする
                        P.append(parsed_line)                                                   #結果を格納
                        host_count(parsed_line.get('remote_host'))                              #ホストを格納
                        jikan(int(parsed_line.get('time_received_isoformat')[14:16]))           #時間を格納
                        day_list.append(int(parsed_line.get('time_received_isoformat')[8:10]))  #日にちを格納
                        month_list.append(int(parsed_line.get('time_received_isoformat')[5:7])) #月を格納
                        year_list.append(int(parsed_line.get('time_received_isoformat')[0:4]))  #年を格納
                    except ValueError:                                                          #アクセスログをパース出来なかった場合
                        E.append(line)                                                          #結果を格納する
        except FileNotFoundError:                                                               #ファイルを選択できなかった場合ループを終了する
            break
    return P

In [4]:
def jikan(j):              #アクセス時間をカウントする関数
    hour[j] = hour[j] + 1

In [5]:
def host_count(remote):    #ホスト名を追加する関数
    host.append(remote)

In [6]:
ifn = 'access_log1'  #ログファイル
read_apache_log(ifn) #ログファイルを読む

[{'remote_host': '10.2.3.5',
  'remote_logname': '-',
  'remote_user': '-',
  'time_received': '[18/Apr/2005:00:11:47 +0900]',
  'time_received_datetimeobj': datetime.datetime(2005, 4, 18, 0, 11, 47),
  'time_received_isoformat': '2005-04-18T00:11:47',
  'time_received_tz_datetimeobj': datetime.datetime(2005, 4, 18, 0, 11, 47, tzinfo='0900'),
  'time_received_tz_isoformat': '2005-04-18T00:11:47+09:00',
  'time_received_utc_datetimeobj': datetime.datetime(2005, 4, 17, 15, 11, 47, tzinfo='0000'),
  'time_received_utc_isoformat': '2005-04-17T15:11:47+00:00',
  'request_first_line': 'GET / HTTP/1.1',
  'request_method': 'GET',
  'request_url': '/',
  'request_http_ver': '1.1',
  'request_url_scheme': '',
  'request_url_netloc': '',
  'request_url_path': '/',
  'request_url_query': '',
  'request_url_fragment': '',
  'request_url_username': None,
  'request_url_password': None,
  'request_url_hostname': None,
  'request_url_port': None,
  'request_url_query_dict': {},
  'request_url_query_l

## (問1)各時間帯毎のアクセス件数を知りたい

In [7]:
def acc_dis():                                 #時間帯別のアクセス件数を表示する
    i=0
    for i in range(len(hour)):
        print(i,"時間のアクセス件数",hour[i])

In [8]:
acc_dis()

0 時間のアクセス件数 0
1 時間のアクセス件数 0
2 時間のアクセス件数 0
3 時間のアクセス件数 0
4 時間のアクセス件数 0
5 時間のアクセス件数 0
6 時間のアクセス件数 0
7 時間のアクセス件数 0
8 時間のアクセス件数 0
9 時間のアクセス件数 0
10 時間のアクセス件数 84
11 時間のアクセス件数 24
12 時間のアクセス件数 36
13 時間のアクセス件数 24
14 時間のアクセス件数 12
15 時間のアクセス件数 12
16 時間のアクセス件数 24
17 時間のアクセス件数 12
18 時間のアクセス件数 12
19 時間のアクセス件数 12
20 時間のアクセス件数 0
21 時間のアクセス件数 0
22 時間のアクセス件数 0
23 時間のアクセス件数 0


## (問1)リモートホスト別のアクセス件数

In [9]:
c = collections.Counter(host)           #ホスト名ごとのアクセス件数を集計する
print(c.most_common())

[('10.2.3.5', 36), ('10.2.3.8', 36), ('10.2.3.9', 36), ('10.2.3.4', 36), ('10.2.3.0', 36), ('10.2.3.7', 24), ('10.2.3.3', 12), ('10.2.3.2', 12), ('10.2.3.1', 12), ('10.2.3.6', 12)]


## (問3) 期間の指定

In [10]:
df = pd.DataFrame({'remote_host' : host, 'year' : year_list, 'month' : month_list, 'day' : day_list})   #ホスト名、年、月、日にちの表示用データフレームを作成する

期間指定する際は年,月,日にちを以下の通りに入力してください。<br>
**年**:西暦で入力してください ex)2000,1999,2019<br>
**月**:入力が一桁の場合、二桁目に0を入力してください　ex)01,06,09<br>
**日にち**:入力が一桁の場合、二桁目に0を入力してください　ex)01,06,09<br>
最後に、年,月,日を入力する場合、間に半角スペースを入力してください。 ex)2000 01 01,1999 06 06<br>

In [11]:
S_year,S_month,S_day = map(int,input('開始＞＞').split())          #表示させたい開始年、月、日にちを入力させる
E_year,E_month,E_day = map(int,input('終了＞＞').split())          #表示させたい終了年、月、日にちを入力させる

開始＞＞ 2005 4 18
終了＞＞ 2006 5 19


In [12]:
df_or = df[(df['year'] >= S_year) & (df['year'] <= E_year) & (df['month'] >= S_month) & (df['month'] <= E_month) & 
           (df['day'] >= S_day) & (df['day'] <= E_day)]    #入力された情報を元に対象となるデータを違うデータフレームに入力する。
df_or.head

<bound method NDFrame.head of     remote_host  year  month  day
0      10.2.3.5  2005      4   18
1      10.2.3.3  2005      4   18
2      10.2.3.2  2005      4   18
3      10.2.3.8  2005      4   18
4      10.2.3.9  2005      4   18
..          ...   ...    ...  ...
247    10.2.3.8  2005      4   18
248    10.2.3.9  2005      4   18
249    10.2.3.7  2005      4   18
250    10.2.3.5  2005      4   18
251    10.2.3.4  2005      4   18

[252 rows x 4 columns]>